In [ ]:
%load_ext autoreload
%autoreload 2   

In [ ]:
import torch
import torch.nn as nn
import networkx as nx
import numpy as np
from icecream import ic

from sklearn.decomposition import PCA
import torch.optim as optim

import random

import matplotlib.pyplot as plt


device = "mps" if torch.backends.mps.is_available() else "cpu"
ic(device);

In [21]:
# Set seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


In [22]:
# Load the Zachary Karate Club graph
G = nx.karate_club_graph()
num_nodes = G.number_of_nodes()


def plot_graph(G, figsize=(10, 7)):
    club_dict = nx.get_node_attributes(G, "club")
    colors = [
        "lightblue" if club_dict[node] == "Mr. Hi" else "red" for node in G.nodes()
    ]
    plt.figure(figsize=figsize)
    nx.draw(
        G,
        pos=nx.spring_layout(G, seed=42),
        with_labels=True,
        node_color=colors,
        edge_color="gray",
        cmap="Set2",
    )


# plot_graph(G, figsize=(5, 5))

![](../resources/biased_rw.png)

in get_alias_edge: 
- src= t
- dst= w

In [ ]:
def get_alias_edge(G, src, dst, p, q):
    """Compute transition probabilities for neighbors of 'dst' based on Node2Vec bias.
    dst: current node
    src: previous node
    p: return back to previous node. Lower p means backtracking is more likely.
    q: in-out param: explore new nodes (ratio of bfs[breadth-first search] to dfs[depth-first search]). 
        lower q means more exploration of distant nodes
    """
    unnormalized_probs = []
    for neighbor in G[dst]:
        if neighbor == src:
            weight = 1.0 / p
        elif G.has_edge(neighbor, src):  # if neighbor is a neighbor of src (above ex: s1 is neighbor of t)
            weight = 1.0
        else:
            weight = 1.0 / q
        unnormalized_probs.append(weight)
    norm_const = sum(unnormalized_probs)
    normalized_probs = [w / norm_const for w in unnormalized_probs]
    return list(G[dst]), normalized_probs


def node2vec_walk(G, start, walk_length, p, q):
    """Generate a random walk starting from the given node."""
    walk = [start]
    while len(walk) < walk_length:
        cur = walk[-1]
        neighbors = list(G.neighbors(cur))
        if len(neighbors) == 0:
            break
        if len(walk) == 1:
            next_node = random.choice(neighbors)
        else:
            prev = walk[-2]
            candidates, probs = get_alias_edge(G, src=prev, dst=cur, p=p, q=q)
            next_node = random.choices(candidates, weights=probs, k=1)[0]
        walk.append(next_node)
    return walk


nodes = list(G.nodes())
node = nodes[0]
ic(node)
for i in range(10):
    walk = node2vec_walk(G, start=node, walk_length=10, p=1, q=1)
    ic(walk)

# decompose walk

![](../resources/zachary.png)

In [24]:
start = 0
walk = [start]
p = 1
q = 1
for i in range(3):
    cur = walk[-1]
    neighbors = list(G.neighbors(cur))
    ic(neighbors)

    if len(neighbors) == 0:
        break
    if len(walk) == 1:
        next_node = random.choice(neighbors)
    else:
        prev = walk[-2]
        candidates, probs = get_alias_edge(G, prev, cur, p, q)
        ic(candidates)
        ic(probs)
        next_node = random.choices(candidates, weights=probs, k=1)[0]
    ic(next_node)
    walk.append(next_node)
    ic(walk)

In [25]:
# Generate random walks for all nodes

# Node2Vec parameters
p = 1.0
q = 1.0
walk_length = 10
num_walks = 10
walks = []
for _ in range(num_walks):
    nodes = list(G.nodes())
    random.shuffle(nodes)
    for node in nodes:
        walk = node2vec_walk(G, node, walk_length, p, q)
        walks.append(walk)


In [26]:
walks[:5]

[[18, 32, 22, 33, 26, 29, 26, 29, 33, 31],
 [20, 33, 15, 32, 22, 32, 29, 33, 22, 33],
 [12, 0, 3, 2, 9, 2, 28, 31, 24, 27],
 [15, 32, 29, 26, 33, 19, 33, 20, 32, 2],
 [28, 2, 9, 2, 3, 0, 12, 0, 21, 1]]

In [29]:
window_size = 2
embedding_dim = 16
num_negative_samples = 5


# A walk represents a sequence of nodes traversed in the graph. 
# For each node (called target) in a walk, the algorithm creates training pairs 
# with surrounding nodes within a window of size window_size.
# window size = 2 means that for each target node,
# the algorithm will consider the two nodes before and two nodes after it in the walk.

def gen_walk_pairs(walk, window_size):
    walk_pairs = []
    for i, target in enumerate(walk):
        start = max(0, i - window_size)
        end = min(len(walk), i + window_size + 1)
        ic(i, target, start, end)
        for j in range(start, end):
            if i != j:
                ic("pair",target, walk[j])
                walk_pairs.append((target, walk[j]))

    return walk_pairs

ic.enable()
w = walks[0]
ic(w)
walk_pairs = gen_walk_pairs(w, window_size)



ic| w: [18, 32, 22, 33, 26, 29, 26, 29, 33, 31]
ic| i: 0, target: 18, start: 0, end: 3
ic| 'pair', target: 18, walk[j]: 32
ic| 'pair', target: 18, walk[j]: 22
ic| i: 1, target: 32, start: 0, end: 4
ic| 'pair', target: 32, walk[j]: 18
ic| 'pair', target: 32, walk[j]: 22
ic| 'pair', target: 32, walk[j]: 33
ic| i: 2, target: 22, start: 0, end: 5
ic| 'pair', target: 22, walk[j]: 18
ic| 'pair', target: 22, walk[j]: 32
ic| 'pair', target: 22, walk[j]: 33
ic| 'pair', target: 22, walk[j]: 26
ic| i: 3, target: 33, start: 1, end: 6
ic| 'pair', target: 33, walk[j]: 32
ic| 'pair', target: 33, walk[j]: 22
ic| 'pair', target: 33, walk[j]: 26
ic| 'pair', target: 33, walk[j]: 29
ic| i: 4, target: 26, start: 2, end: 7
ic| 'pair', target: 26, walk[j]: 22
ic| 'pair', target: 26, walk[j]: 33
ic| 'pair', target: 26, walk[j]: 29
ic| 'pair', target: 26, walk[j]: 26
ic| i: 5, target: 29, start: 3, end: 8
ic| 'pair', target: 29, walk[j]: 33
ic| 'pair', target: 29, walk[j]: 26
ic| 'pair', target: 29, walk[j]: 2

In [30]:
ic.disable()
pairs = []
for walk in walks:
    walk_pairs = gen_walk_pairs(walk, window_size)
    pairs.extend(walk_pairs)

pairs = np.array(pairs)


![](../resources/loss_rw.png)

In [36]:
ic.enable()
ic(num_negative_samples)
negative_batch = torch.LongTensor(
    np.random.randint(0, num_nodes, size=(len(batch), num_negative_samples))
).to(device)
ic(negative_batch);

ic| num_negative_samples: 5
ic| negative_batch: tensor([[11, 15,  3, 25, 24],
                            [17, 22, 23, 21, 10],
                            [26, 12, 22,  5, 20],
                            [29, 32,  1,  8, 13],
                            [28, 33, 18,  2, 27],
                            [17, 21,  1, 33, 18],
                            [10, 30, 21, 26,  1],
                            [10, 27,  7, 30, 22],
                            [29, 27, 14, 28,  4],
                            [ 8,  5,  5,  2, 32],
                            [ 9, 28,  0,  3,  3],
                            [27, 21, 14, 31, 10],
                            [26, 16, 17, 23, 30],
                            [29,  4, 27, 30, 18],
                            [16, 28, 30, 25, 20],
                            [ 5, 12, 22, 11, 19],
                            [33, 16, 19, 32, 30],
                            [20, 28, 20, 11,  8],
                            [ 3,  4, 10,  6,  0],
                      

In [66]:
# Define the Node2Vec Skip-Gram model
class Node2Vec(nn.Module):
    def __init__(self, num_nodes, embedding_dim):
        super(Node2Vec, self).__init__()
        self.in_embeddings = nn.Embedding(num_nodes, embedding_dim)
        self.out_embeddings = nn.Embedding(num_nodes, embedding_dim)
        nn.init.xavier_uniform_(self.in_embeddings.weight)
        nn.init.xavier_uniform_(self.out_embeddings.weight)

    def forward(self, target, context, negative):
        ic(target.shape, context.shape, negative.shape)
        embed_target = self.in_embeddings(target)  # (batch, embedding_dim)
        embed_context = self.out_embeddings(context)  # (batch, embedding_dim)
        score = torch.sum(embed_target * embed_context, dim=1)
        log_target = torch.log(torch.sigmoid(score) + 1e-10)

        embed_negative = self.out_embeddings(
            negative
        )  # (batch, num_negative, embedding_dim)
        neg_score = torch.bmm(embed_negative, embed_target.unsqueeze(2)).squeeze()
        log_negative = torch.sum(torch.log(torch.sigmoid(-neg_score) + 1e-10), dim=1)

        loss = -(log_target + log_negative)
        return loss.mean()


model = Node2Vec(num_nodes, embedding_dim).to(device)

model.forward(target=torch.LongTensor([0,1]).to(device), 
      context=torch.LongTensor([1,2]).to(device),
      negative=torch.LongTensor([[3,2],[4,6]]).to(device))


ic| target.shape: torch.Size([2])
    context.shape: torch.Size([2])
    negative.shape: torch.Size([2, 2])


tensor(2.1213, device='mps:0', grad_fn=<MeanBackward0>)

In [68]:
model = Node2Vec(num_nodes, embedding_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
# Training loop
num_epochs = 100
batch_size = 64
num_batches = len(pairs) // batch_size + 1
ic.disable()
for epoch in range(num_epochs):
    np.random.shuffle(pairs)
    epoch_loss = 0
    for i in range(0, len(pairs), batch_size):
        batch = pairs[i : i + batch_size]
        ic(batch.shape,batch[:,0].shape)
        target_batch = torch.LongTensor(batch[:, 0]).to(device)
        context_batch = torch.LongTensor(batch[:, 1]).to(device)
        # Nb: there is no guarantee that the negative samples are actually negative (not connected to target nodes)
        # to be perfectly negative, For each target node, sample from nodes that are not its neighbors
        negative_batch = torch.LongTensor(
            np.random.randint(0, num_nodes, size=(len(batch), num_negative_samples))
        ).to(device)

        optimizer.zero_grad()
        loss = model(target_batch, context_batch, negative_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss / num_batches:.4f}")


Epoch 10, Loss: 1.9572
Epoch 20, Loss: 1.9548
Epoch 30, Loss: 1.9507
Epoch 40, Loss: 1.9668
Epoch 50, Loss: 1.9616
Epoch 60, Loss: 1.9580
Epoch 70, Loss: 1.9565
Epoch 80, Loss: 1.9482
Epoch 90, Loss: 1.9579
Epoch 100, Loss: 1.9583


In [ ]:

# Retrieve learned embeddings
embeddings = model.in_embeddings.weight.data.cpu().numpy()
print("\nLearned embeddings:")
for node in range(num_nodes):
    print(f"Node {node}: {embeddings[node]}")

# Use PCA to reduce the embeddings to 2D for visualization
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings)

# Create colors list based on club membership: red for 'Mr. Hi', blue for others.
colors = []
for node in range(num_nodes):
    club = G.nodes[node]["club"]
    colors.append("red" if club == "Mr. Hi" else "blue")

# Plotting the PCA visualization
plt.figure(figsize=(8, 6))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=colors)
for i in range(num_nodes):
    plt.annotate(str(i), (embeddings_2d[i, 0], embeddings_2d[i, 1]), fontsize=9)
plt.title("PCA Visualization of Node2Vec Embeddings")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.show()

## gemini

Solution suggested by gemini

In [ ]:
import torch
import networkx as nx
import matplotlib.pyplot as plt
import random
from collections import defaultdict
from tqdm import tqdm


# Load the Zachary Karate Club graph
def load_karate_graph():
    """
    Loads the Zachary Karate Club graph from NetworkX.

    Returns:
        nx.Graph: The Karate Club graph.
    """
    G = nx.karate_club_graph()
    return G


# Convert NetworkX graph to adjacency list
def graph_to_adj_list(graph):
    """
    Converts a NetworkX graph to an adjacency list representation.

    Args:
        graph (nx.Graph): The input graph.

    Returns:
        dict: An adjacency list where keys are nodes and values are lists of neighbors.
    """
    adj_list = defaultdict(list)
    for node in graph.nodes():
        adj_list[node] = list(graph.neighbors(node))
    return adj_list


# Generate random walk
def generate_random_walk(graph, start_node, walk_length, p, q):
    """
    Generates a random walk starting from a given node.  This implementation
    correctly handles the edge weights (p and q) for the walk.

    Args:
        graph (nx.Graph): The input graph.
        start_node (int): The starting node for the random walk.
        walk_length (int): The length of the random walk.
        p (float): The return parameter.
        q (float): The in-out parameter.

    Returns:
        list: A list of nodes representing the random walk.
    """
    walk = [start_node]
    for _ in range(walk_length - 1):
        current_node = walk[-1]
        neighbors = list(graph.neighbors(current_node))
        if not neighbors:
            break  # Handle disconnected nodes
        if len(walk) == 1:
            # Start node case: sample directly from neighbors
            next_node = random.choice(neighbors)
            walk.append(next_node)
        else:
            # Subsequent step case: consider p and q
            previous_node = walk[-2]
            probabilities = []
            for neighbor in neighbors:
                if neighbor == previous_node:
                    probabilities.append(1 / p)  # Return probability
                elif graph.has_edge(previous_node, neighbor):
                    probabilities.append(1)  # Neighboring node
                else:
                    probabilities.append(1 / q)  # Non-neighboring node
            # Normalize probabilities to sum to 1
            probabilities = [prob / sum(probabilities) for prob in probabilities]
            # Use the probabilities to select the next node.
            next_node = random.choices(neighbors, probabilities)[0]
            walk.append(next_node)
    return walk


# Generate multiple random walks
def generate_walks(graph, num_walks, walk_length, p, q):
    """
    Generates multiple random walks for each node in the graph.

    Args:
        graph (nx.Graph): The input graph.
        num_walks (int): The number of random walks per node.
        walk_length (int): The length of each random walk.
        p (float): The return parameter.
        q (float): The in-out parameter.

    Returns:
        list: A list of lists, where each inner list is a random walk.
    """
    walks = []
    nodes = list(graph.nodes())
    for _ in range(num_walks):
        random.shuffle(nodes)  # Shuffle for each iteration
        for node in nodes:
            walk = generate_random_walk(graph, node, walk_length, p, q)
            walks.append(walk)
    return walks


# Skip-gram model with negative sampling
class SkipGramModel(torch.nn.Module):
    """
    Skip-gram model for learning node embeddings.  This implementation
    uses PyTorch and includes proper initialization.
    """

    def __init__(self, num_nodes, embedding_dim):
        """
        Initializes the Skip-gram model.

        Args:
            num_nodes (int): The number of nodes in the graph.
            embedding_dim (int): The dimensionality of the node embeddings.
        """
        super(SkipGramModel, self).__init__()
        self.embedding_u = torch.nn.Embedding(
            num_nodes, embedding_dim
        )  # Target embeddings
        self.embedding_v = torch.nn.Embedding(
            num_nodes, embedding_dim
        )  # Context embeddings

        # Initialize embeddings using a small uniform distribution
        init_range = 0.5 / embedding_dim
        self.embedding_u.weight.data.uniform_(-init_range, init_range)
        self.embedding_v.weight.data.uniform_(
            -0, 0
        )  # init_range, init_range) # Initializing to 0 can sometimes help

    def forward(self, center_nodes, context_nodes, negative_samples):
        """
        Computes the forward pass of the Skip-gram model.

        Args:
            center_nodes (torch.Tensor): Tensor of center node indices.
            context_nodes (torch.Tensor): Tensor of context node indices.
            negative_samples (torch.Tensor): Tensor of negative sample indices.

        Returns:
            torch.Tensor: The loss value.
        """
        # Get embeddings for center nodes
        u_embeddings = self.embedding_u(center_nodes)  # (batch_size, embedding_dim)

        # Get embeddings for context nodes
        v_embeddings = self.embedding_v(context_nodes)  # (batch_size, embedding_dim)

        # Compute the positive log-likelihood
        positive_scores = torch.sum(u_embeddings * v_embeddings, dim=1)  # (batch_size,)
        positive_loss = -torch.mean(torch.log(torch.sigmoid(positive_scores)))

        # Get embeddings for negative samples
        negative_embeddings = self.embedding_v(
            negative_samples
        )  # (batch_size, num_negative_samples, embedding_dim)

        # Compute the negative log-likelihood
        negative_scores = torch.bmm(
            negative_embeddings, u_embeddings.unsqueeze(2)
        ).squeeze(2)  # (batch_size, num_negative_samples)
        negative_loss = -torch.mean(torch.log(torch.sigmoid(-negative_scores)))

        # Combine positive and negative losses
        loss = positive_loss + negative_loss
        return loss


def generate_negative_samples(walks, num_nodes, window_size, num_negative_samples):
    """
    Generates negative samples for each center node and context node pair.
    This version pre-computes the unigram distribution and samples directly
    according to that distribution.

    Args:
        walks (list): A list of random walks.
        num_nodes (int): The total number of nodes in the graph.
        window_size (int): The context window size.
        num_negative_samples (int): The number of negative samples per (center, context) pair.

    Returns:
        list: A list of tuples, where each tuple contains:
            (center_node, context_node, list of negative samples)
    """
    negative_samples = []
    node_counts = [0] * num_nodes  # Initialize node counts
    for walk in walks:
        for node in walk:
            node_counts[node] += 1

    # Compute the unigram distribution raised to the power of 0.75
    total_count = sum(node_counts)
    unigram_dist = [(count / total_count) ** 0.75 for count in node_counts]
    # Normalize the distribution
    unigram_dist = [p / sum(unigram_dist) for p in unigram_dist]

    for walk in walks:
        for i, center_node in enumerate(walk):
            for j in range(
                max(0, i - window_size), min(len(walk), i + window_size + 1)
            ):
                if i != j:
                    context_node = walk[j]
                    # Generate negative samples, ensuring they are different from the context node
                    neg_samples = random.choices(
                        range(num_nodes), weights=unigram_dist, k=num_negative_samples
                    )
                    negative_samples.append((center_node, context_node, neg_samples))
    return negative_samples


def train_node2vec(
    graph,
    walks,
    embedding_dim,
    window_size,
    num_negative_samples,
    batch_size,
    epochs,
    learning_rate,
    p,
    q,
    num_workers=0,
):
    """
    Trains the Node2vec model. This function incorporates the negative sampling
    and training loop, and uses the graph, walks, and p, q parameters.

    Args:
        graph (nx.Graph): The input graph.
        walks (list): A list of random walks.
        embedding_dim (int): The dimensionality of the node embeddings.
        window_size (int): The context window size.
        num_negative_samples (int): The number of negative samples per (center, context) pair.
        batch_size (int): The batch size for training.
        epochs (int): The number of training epochs.
        learning_rate (float): The learning rate.
        p (float): The return parameter.
        q (float): The in-out parameter.
        num_workers (int): Number of workers for data loading (not used in this basic implementation).

    Returns:
        torch.nn.Module: The trained Skip-gram model.
    """
    # Determine the device to use
    device = (
        torch.device("mps")
        if torch.backends.mps.is_available()
        else torch.device("cpu")
    )
    print(f"Using device: {device}")

    num_nodes = graph.number_of_nodes()
    model = SkipGramModel(num_nodes, embedding_dim).to(device)  # Move model to device
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    # Use Binary Cross Entropy Loss (though we calculate it manually for more control)
    # criterion = torch.nn.BCEWithLogitsLoss() # Not used, but kept here for reference

    negative_samples = generate_negative_samples(
        walks, num_nodes, window_size, num_negative_samples
    )

    # Prepare data for PyTorch (create tensors) outside the loop
    center_nodes_all = []
    context_nodes_all = []
    negative_samples_all = []

    for center_node, context_node, neg_samples in negative_samples:
        center_nodes_all.append(center_node)
        context_nodes_all.append(context_node)
        negative_samples_all.append(neg_samples)

    center_nodes_tensor = torch.tensor(center_nodes_all, dtype=torch.long).to(
        device
    )  # Move tensors to device
    context_nodes_tensor = torch.tensor(context_nodes_all, dtype=torch.long).to(device)
    negative_samples_tensor = torch.tensor(negative_samples_all, dtype=torch.long).to(
        device
    )

    num_samples = len(negative_samples)
    for epoch in range(epochs):
        # Shuffle the training data at the beginning of each epoch
        permutation = torch.randperm(num_samples).to(
            device
        )  # Move permutation to device
        center_nodes_tensor = center_nodes_tensor[permutation]
        context_nodes_tensor = context_nodes_tensor[permutation]
        negative_samples_tensor = negative_samples_tensor[permutation]
        total_loss = 0.0
        for i in tqdm(
            range(0, num_samples, batch_size), desc=f"Epoch {epoch + 1}/{epochs}"
        ):
            optimizer.zero_grad()
            # Get batch
            indices = torch.arange(i, min(i + batch_size, num_samples)).to(
                device
            )  # Move indices to device
            batch_center_nodes = center_nodes_tensor[indices]
            batch_context_nodes = context_nodes_tensor[indices]
            batch_negative_samples = negative_samples_tensor[indices]

            # Forward pass
            loss = model(
                batch_center_nodes, batch_context_nodes, batch_negative_samples
            )
            total_loss += loss.item()
            # Backward pass
            loss.backward()
            optimizer.step()
        print(
            f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / (num_samples // batch_size)}"
        )
    return model


def get_node_embeddings(model):
    """
    Gets the node embeddings from the trained model.

    Args:
        model (torch.nn.Module): The trained Skip-gram model.

    Returns:
        torch.Tensor: A tensor containing the node embeddings.
    """
    return model.embedding_u.weight.data.cpu().numpy()


def visualize_embeddings(embeddings, graph, title="Node2vec Embeddings"):
    """
    Visualizes the node embeddings using a 2D scatter plot.  This version
    uses the actual node labels from the Karate Club graph.

    Args:
        embeddings (numpy.ndarray): The node embeddings.
        graph (nx.Graph): The input graph (for getting node labels).
        title (str): The title of the plot.
    """
    # Use a simple PCA for dimensionality reduction to 2D
    from sklearn.decomposition import PCA

    pca = PCA(n_components=2)
    reduced_embeddings = pca.fit_transform(embeddings)

    # Get the club labels for coloring
    club_labels = [graph.nodes[node]["club"] for node in graph.nodes()]
    colors = ["red" if label == "Mr. Hi" else "blue" for label in club_labels]

    plt.figure(figsize=(10, 8))
    plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c=colors)

    # Add node labels to the plot
    for i, node in enumerate(graph.nodes()):
        plt.annotate(
            str(node),
            xy=(reduced_embeddings[i, 0], reduced_embeddings[i, 1]),
            xytext=(5, 2),
            textcoords="offset points",
            fontsize=10,
        )

    plt.title(title)
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.grid(True)
    plt.show()


if __name__ == "__main__":
    # Hyperparameters
    embedding_dim = 128
    walk_length = 20
    num_walks = 10
    window_size = 5
    num_negative_samples = 5
    batch_size = 32
    epochs = 50
    learning_rate = 0.05  # Increased learning rate
    p = 1.0
    q = 1.0

    # Load graph
    karate_graph = load_karate_graph()

    # Generate random walks
    walks = generate_walks(karate_graph, num_walks, walk_length, p, q)

    # Train Node2vec model
    model = train_node2vec(
        karate_graph,
        walks,
        embedding_dim,
        window_size,
        num_negative_samples,
        batch_size,
        epochs,
        learning_rate,
        p,
        q,
    )

    # Get node embeddings
    node_embeddings = get_node_embeddings(model)

    # Visualize embeddings
    visualize_embeddings(
        node_embeddings, karate_graph, title="Node2vec Embeddings (Karate Club)"
    )

    # Print the first 5 embeddings
    print("First 5 Node Embeddings:")
    print(node_embeddings[:5])
